In [10]:
import requests
import dotenv
import os
dotenv.load_dotenv()  # .env 환경변수 파일을 로드함

key = os.getenv("SEOUL_SUBWAY_API")

In [11]:
# 지하철 실시간 도착정보  https://data.seoul.go.kr/dataList/OA-12764/F/1/datasetView.do
# http://swopenAPI.seoul.go.kr/api/subway/(인증키)/xml/realtimeStationArrival/0/5/서울

url = f'http://swopenAPI.seoul.go.kr/api/subway/{key}/json/realtimeStationArrival/0/100/강남'
response = requests.get(url)
response.status_code

200

In [12]:
respnse = response.json()
for row in respnse['realtimeArrivalList']:
    print(f'열차종류:{row["btrainSttus"]} 도착예정시간:{int(row["barvlDt"])//60}분 {int(row["barvlDt"])%60}초')
    print(row['trainLineNm'],row['statnNm'],row['statnFid'],row['statnId'],row['statnTid'])
    print('*'*100)   

열차종류:일반 도착예정시간:1분 30초
성수행 - 역삼방면 강남 1002000223 1002000222 1002000221
****************************************************************************************************
열차종류:일반 도착예정시간:0분 0초
신사행 - 신논현방면 강남 1077000688 1077000687 1077000686
****************************************************************************************************
열차종류:일반 도착예정시간:4분 30초
성수행 - 역삼방면 강남 1002000223 1002000222 1002000221
****************************************************************************************************
열차종류:일반 도착예정시간:0분 0초
신사행 - 신논현방면 강남 1077000688 1077000687 1077000686
****************************************************************************************************
열차종류:일반 도착예정시간:0분 0초
광교행 - 양재방면 강남 1077000686 1077000687 1077000688
****************************************************************************************************
열차종류:일반 도착예정시간:3분 50초
성수행 - 교대방면 강남 1002000221 1002000222 1002000223
**********************************************************************************

In [13]:
line2_stations = [
    # 본선
    "시청", "을지로입구", "을지로3가", "을지로4가", "동대문역사문화공원", "신당", "상왕십리",
    "왕십리", "한양대", "뚝섬", "성수", "건대입구", "구의", "강변", "잠실나루", "잠실",
    "잠실새내", "종합운동장", "삼성", "선릉", "역삼", "강남", "교대", "서초", "방배", 
    "사당", "낙성대", "서울대입구", "봉천", "신림", "신대방", "구로디지털단지", "대림",
    "신도림", "문래", "영등포구청", "당산", "합정", "홍대입구", "신촌", "이대", "아현",
    "충정로", "시청",
    # 성수지선
    "성수", "용답", "신답", "용두", "신설동",
    # 신정지선
    "신도림", "도림천", "양천구청", "신정네거리", "까치산"
]

In [ ]:
# 역명 - 역명코드  매핑테이블 만들기
mappings = {}
for station in line2_stations:
    url = f'http://swopenAPI.seoul.go.kr/api/subway/{key}/json/realtimeStationArrival/0/100/{station}'
    response = requests.get(url)    
    respnse = response.json()
    values = []
    for row in respnse['realtimeArrivalList']:                        
        values.append( row['statnId'] )
    mappings[station] = set(values)
mappings 


In [15]:
search_id = '1092004713'
finded_stname =  [ key for key,values in mappings.items() if search_id in values ]
result = finded_stname[0] if finded_stname else None
print(result)

신설동


In [16]:
# 매핑 함수(역코드 -> 역명)
# 2호선기준으로 실시간 도착 역명을 입력 -> 노선, 이전역명, 다음역명, 도착예정시간을 알려주는 코드

In [17]:
# 매핑함수  search_id = '1092004713'
def find_station_name(search_id):    
    finded_stname =  [ key for key,values in mappings.items() if search_id in values ]
    return finded_stname[0] if finded_stname else None  

In [22]:
url = f'http://swopenAPI.seoul.go.kr/api/subway/{key}/json/realtimeStationArrival/0/100/낙성대'
response = requests.get(url).json()
for row in response['realtimeArrivalList']:
    print(f'열차종류:{row["btrainSttus"]} 도착예정시간:{int(row["barvlDt"])//60}분 {int(row["barvlDt"])%60}초')
    bst = find_station_name( row['statnFid'] )
    ast = find_station_name( row['statnTid'] )
    print(f"{row['trainLineNm']},이전역:{bst},다음역:{ast}")
    print('*'*100)           

열차종류:일반 도착예정시간:2분 30초
성수행 - 사당방면,이전역:서울대입구,다음역:사당
****************************************************************************************************
열차종류:일반 도착예정시간:7분 40초
성수행 - 사당방면,이전역:서울대입구,다음역:사당
****************************************************************************************************
열차종류:일반 도착예정시간:2분 0초
성수행 - 서울대입구방면,이전역:사당,다음역:서울대입구
****************************************************************************************************
열차종류:일반 도착예정시간:6분 40초
성수행 - 서울대입구방면,이전역:사당,다음역:서울대입구
****************************************************************************************************
